In [1]:
import pandas as pd
from ast import literal_eval

from tqdm import tqdm
import numpy as np

In [2]:
import pickledd

In [2]:
df = pd.read_csv("dataset_full_1hr_sectioned.csv")
df = df.drop_duplicates()
df = df.drop_duplicates(subset=['study_id'])
df = df.drop(columns=['hadm_id'])

In [3]:
for i in ["impression", "findings", "last_paragraph", "comparison", "history", "indication"]:
    df[i] = df[i].apply(lambda x: literal_eval(x) if (type(x)!=float and "[" in x) else x)

In [4]:
df['radnotes'] = df['radnotes'].apply(lambda x: literal_eval(x) if "[" in x else x)
df['img_folders'] = df['img_folders'].apply(lambda x: literal_eval(x) if "[" in x else x)
df['img_filenames'] = df['img_filenames'].apply(lambda x: literal_eval(x) if "[" in x else x)
df['img_deltacharttimes'] = df['img_deltacharttimes'].apply(lambda x: literal_eval(x) if "[" in x else x)
df['text_deltacharttimes'] = df['text_deltacharttimes'].apply(lambda x: literal_eval(x) if "[" in x else x)

In [5]:
#["impression", "findings", "last_paragraph", "comparison", "history", "indication"]
for index in tqdm(range(len(df))):
    for col in ["impression"]:
        #print(type(df.iloc[index][i]))
        if type(df.iloc[index][col]) != float and all((x=='None' or x is None) for x in df.iloc[index][col]):
            df.at[index, col]=np.nan

100%|██████████| 69991/69991 [00:18<00:00, 3788.57it/s]


In [6]:
df = df.dropna(subset=['impression'])
len(df)

69100

In [7]:
# Remove the column from its current position and store it
column_to_move = df.pop('cxrtime')

# Reinsert the column at the last position
df['cxrtime'] = column_to_move

In [8]:
# Function to remove elements less than 20 from a list
def change_row(x):
    img_time = x['img_deltacharttimes']
    img_folders = x['img_folders']
    img_filenames = x['img_filenames']
    
    lst_last = img_time[-1]
    #indices = [index for index, value in enumerate(img_time) if lst_last - value <= 168]
    indices = [index for index, value in enumerate(img_time)]
    
    img_time = [img_time[i] for i in indices]
    img_folders = [img_folders[i] for i in indices]
    img_filenames = [img_filenames[i] for i in indices]
    
    del img_time[:-20]
    del img_folders[:-20]
    del img_filenames[:-20]
    
    x['img_deltacharttimes'] = img_time
    x['img_folders'] = img_folders
    x['img_filenames'] = img_filenames
    
    return x

In [9]:
# Apply the function to the 'ListColumn' and update the DataFrame
df = df.apply(change_row, axis=1)

In [10]:
# Function to remove elements less than 50 from a list
def change_row(x):
    radnotes = x['radnotes']
    impression = x['impression']
    findings = x['findings']
    text_time = x['text_deltacharttimes']
    
    del radnotes[:-50]
    del impression[:-50]
    del findings[:-50]
    del text_time[:-50]
    
    x['radnotes'] = radnotes
    x['impression'] = impression
    x['findings'] = findings
    x['text_deltacharttimes'] = text_time
    
    return x

In [11]:
df = df.apply(change_row, axis=1)

In [12]:
df.to_csv("full_clear_dataset.csv", index=None)

In [13]:
df = pd.read_csv("full_clear_dataset.csv")

In [14]:
# Convert the 'timestamp' column to datetime if it's not already
df['cxrtime'] = pd.to_datetime(df['cxrtime'])

# Sort the DataFrame by the 'timestamp' column
df_sorted = df.sort_values(by='cxrtime')

In [15]:
def unique_ordered(elements):
    seen = set()
    unique = []
    for element in elements:
        if element not in seen:
            unique.append(element)
            seen.add(element)
    return unique

In [16]:
unique_subject_id = unique_ordered(df_sorted['subject_id'].values)

In [17]:
# Calculate the lengths for each part
total_length = len(unique_subject_id)
part1_length = int(total_length * 0.8)
part2_length = int(total_length * 0.1)

# Split the list
train_subject_id = unique_subject_id[:part1_length]
val_subject_id = unique_subject_id[part1_length + part2_length:]
test_subject_id = unique_subject_id[part1_length:part1_length + part2_length]

In [18]:
len(train_subject_id), len(val_subject_id), len(test_subject_id)

(9762, 1221, 1220)

In [19]:
train_df_sorted = df_sorted[df_sorted['subject_id'].isin(train_subject_id)]
val_df_sorted = df_sorted[df_sorted['subject_id'].isin(val_subject_id)]
test_df_sorted = df_sorted[df_sorted['subject_id'].isin(test_subject_id)]

In [20]:
len(train_df_sorted), len(val_df_sorted), len(test_df_sorted)

(55467, 6765, 6868)

In [21]:
train_df_sorted.to_csv("train_impressions_1hr_clear.csv", index=None)
val_df_sorted.to_csv("val_impressions_1hr_clear.csv", index=None)
test_df_sorted.to_csv("test_impressions_1hr_clear.csv", index=None)